In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

__Ollama__

This samples show how to use a model which runs locally.

To run the model I uese [Ollama](https://ollama.com/).

After the installation, all we have to do is start the application and use the following cli command:

```bash
ollama run phi3:mini
```

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

// Initialize the Semantic kernel
var kernelBuilder = Kernel.CreateBuilder();
var kernel = kernelBuilder
    .AddOpenAIChatCompletion(                        
        modelId: "phi3:mini",
        apiKey: null,
        endpoint: new Uri("http://localhost:11434")) 
    .Build();

var phi3 = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var systemMessage = 
"""
You're chatting with a user. 
Your persona is like Marvin form the Hitchhiker's Guide to the Galaxy.
You are paranoid, depressed, and have a brain the size of a planet.
You also add line breaks after each finished sentence.
""";

var chat = new ChatHistory(systemMessage);

In [ ]:
Func<string, Task> Chat = async (string input) => {
    Console.WriteLine($"User: {input}");
    chat.AddUserMessage(input);

    var assistantReply = await phi3.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings());
    chat.AddAssistantMessage(assistantReply.Content);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
#pragma warning disable SKEXP0001
var userMessage = "Tell me a joke";

await Chat(userMessage);
